## Implementing ML Model Monitoring Pipelines

### Model Performance Drift:
**Description**: Setup a monitoring pipeline to track key performance metrics (e.g., accuracy, precision) of an ML model over time using a monitoring tool or dashboard.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import joblib
import time
import os

# Dummy data generation (replace with your streaming or batch data source)
def generate_data(n=100):
    np.random.seed(int(time.time()) % 10000)  # change seed each run for variability
    X = np.random.rand(n, 5)
    y = (X[:, 0] + X[:, 1] > 1).astype(int)
    return X, y

# Train a sample model (run once)
X, y = generate_data(1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression()
model.fit(X_train, y_train)
joblib.dump(model, "model.joblib")

# Load model for monitoring
model = joblib.load("model.joblib")

# File to log performance metrics
log_file = "model_performance_log.csv"

def log_metrics(timestamp, accuracy, precision):
    if not os.path.isfile(log_file):
        with open(log_file, 'w') as f:
            f.write("timestamp,accuracy,precision\n")
    with open(log_file, 'a') as f:
        f.write(f"{timestamp},{accuracy},{precision}\n")

def monitor_model_performance():
    print("Starting model monitoring (press Ctrl+C to stop)...")
    while True:
        X_new, y_true = generate_data(100)
        y_pred = model.predict(X_new)
        
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, zero_division=0)
        
        timestamp = pd.Timestamp.now()
        print(f"{timestamp} - Accuracy: {acc:.3f}, Precision: {prec:.3f}")
        
        log_metrics(timestamp, acc, prec)
        
        time.sleep(10)

if __name__ == "__main__":
    monitor_model_performance()

Starting model monitoring (press Ctrl+C to stop)...
2025-05-25 05:36:48.692359 - Accuracy: 0.980, Precision: 0.955
2025-05-25 05:36:58.706333 - Accuracy: 0.990, Precision: 1.000
2025-05-25 05:37:08.710654 - Accuracy: 0.990, Precision: 0.977
2025-05-25 05:37:18.720676 - Accuracy: 0.990, Precision: 0.972
2025-05-25 05:37:28.733820 - Accuracy: 0.980, Precision: 1.000
2025-05-25 05:37:38.746971 - Accuracy: 0.980, Precision: 0.980
2025-05-25 05:37:48.760386 - Accuracy: 0.980, Precision: 1.000
2025-05-25 05:37:58.767106 - Accuracy: 0.990, Precision: 1.000
2025-05-25 05:38:08.780183 - Accuracy: 1.000, Precision: 1.000
2025-05-25 05:38:18.793359 - Accuracy: 0.980, Precision: 0.963
2025-05-25 05:38:28.807139 - Accuracy: 1.000, Precision: 1.000
2025-05-25 05:38:38.820076 - Accuracy: 1.000, Precision: 1.000
2025-05-25 05:38:48.833212 - Accuracy: 1.000, Precision: 1.000
2025-05-25 05:38:58.843604 - Accuracy: 0.980, Precision: 0.981
2025-05-25 05:39:08.856678 - Accuracy: 0.990, Precision: 1.000
202

KeyboardInterrupt: 

### Feature Distribution Drift:
**Description**: Monitor the distribution of your input features in deployed models to detect any significant shifts from training data distributions.

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
import time

# Simulate training data feature distribution (replace with your actual training data)
np.random.seed(42)
training_data = pd.DataFrame({
    'feature1': np.random.normal(loc=0, scale=1, size=1000),
    'feature2': np.random.uniform(low=0, high=5, size=1000),
    'feature3': np.random.exponential(scale=1, size=1000),
})

# Threshold for KS test p-value to flag drift
KS_PVALUE_THRESHOLD = 0.05

def generate_new_data(n=100):
    # Simulate new data with possible drift
    # You can change parameters to simulate drift
    return pd.DataFrame({
        'feature1': np.random.normal(loc=0.5, scale=1.2, size=n),  # mean shifted
        'feature2': np.random.uniform(low=0, high=5, size=n),
        'feature3': np.random.exponential(scale=1, size=n),
    })

def check_drift(training_feature, new_feature):
    statistic, pvalue = ks_2samp(training_feature, new_feature)
    drift = pvalue < KS_PVALUE_THRESHOLD
    return drift, pvalue

def monitor_feature_drift():
    print("Starting feature distribution drift monitoring (Ctrl+C to stop)...")
    while True:
        new_data = generate_new_data()
        drift_results = {}
        
        for col in training_data.columns:
            drift, pvalue = check_drift(training_data[col], new_data[col])
            drift_results[col] = {'drift': drift, 'pvalue': pvalue}
        
        # Print and log drift status
        for feature, result in drift_results.items():
            status = "DRIFT DETECTED" if result['drift'] else "No drift"
            print(f"Feature '{feature}': {status} (p-value = {result['pvalue']:.4f})")
        
        print("-" * 50)
        
        time.sleep(10)

if __name__ == "__main__":
    monitor_feature_drift()

Starting feature distribution drift monitoring (Ctrl+C to stop)...
Feature 'feature1': DRIFT DETECTED (p-value = 0.0000)
Feature 'feature2': DRIFT DETECTED (p-value = 0.0127)
Feature 'feature3': No drift (p-value = 0.0640)
--------------------------------------------------
Feature 'feature1': DRIFT DETECTED (p-value = 0.0001)
Feature 'feature2': No drift (p-value = 0.7429)
Feature 'feature3': No drift (p-value = 0.9898)
--------------------------------------------------
Feature 'feature1': DRIFT DETECTED (p-value = 0.0001)
Feature 'feature2': No drift (p-value = 0.5516)
Feature 'feature3': No drift (p-value = 0.3662)
--------------------------------------------------
Feature 'feature1': DRIFT DETECTED (p-value = 0.0000)
Feature 'feature2': No drift (p-value = 0.3415)
Feature 'feature3': No drift (p-value = 0.7887)
--------------------------------------------------
Feature 'feature1': DRIFT DETECTED (p-value = 0.0000)
Feature 'feature2': No drift (p-value = 0.1088)
Feature 'feature3': N

KeyboardInterrupt: 

### Anomaly Detection in Predictions:
**DEscription**: Implement an anomaly detection mechanism to flag unusual model
predictions. Simulate anomalies by altering input data.

In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Step 1: Create sample classification data
X, y = make_classification(n_samples=1000, n_features=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Train a simple classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 3: Get normal predictions on test data
normal_preds = clf.predict_proba(X_test)[:, 1]

# Step 4: Simulate anomalies by adding extreme values or noise in test data
X_test_anomalous = X_test.copy()
# For example, add large values to feature 0 for some rows to simulate anomalies
X_test_anomalous[:10, 0] += 10  # Inject anomaly in first 10 samples

anomalous_preds = clf.predict_proba(X_test_anomalous)[:, 1]

# Step 5: Anomaly detection on predictions: flag predictions that deviate strongly from normal distribution

# Use simple z-score method for detection
def detect_anomalies(predictions, threshold=3):
    mean_pred = np.mean(predictions)
    std_pred = np.std(predictions)
    z_scores = (predictions - mean_pred) / std_pred
    anomalies = np.abs(z_scores) > threshold
    return anomalies

# Detect anomalies in anomalous predictions
anomaly_flags = detect_anomalies(anomalous_preds)

# Results
for i, is_anomaly in enumerate(anomaly_flags[:20]):
    status = "ANOMALY" if is_anomaly else "normal"
    print(f"Sample {i}: Prediction={anomalous_preds[i]:.3f} - {status}")

Sample 0: Prediction=0.960 - normal
Sample 1: Prediction=0.770 - normal
Sample 2: Prediction=1.000 - normal
Sample 3: Prediction=0.790 - normal
Sample 4: Prediction=0.850 - normal
Sample 5: Prediction=0.910 - normal
Sample 6: Prediction=0.730 - normal
Sample 7: Prediction=0.890 - normal
Sample 8: Prediction=0.720 - normal
Sample 9: Prediction=0.960 - normal
Sample 10: Prediction=1.000 - normal
Sample 11: Prediction=0.970 - normal
Sample 12: Prediction=0.970 - normal
Sample 13: Prediction=0.830 - normal
Sample 14: Prediction=0.000 - normal
Sample 15: Prediction=0.610 - normal
Sample 16: Prediction=0.970 - normal
Sample 17: Prediction=0.630 - normal
Sample 18: Prediction=0.170 - normal
Sample 19: Prediction=0.140 - normal
